# Modello categoriale {#sec-stan-categorical}

**Prerequisiti**

**Concetti e Competenze Chiave**

**Preparazione del Notebook**

In [4]:
import logging
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import requests
import cmdstanpy
from cmdstanpy import CmdStanModel
cmdstanpy.utils.get_logger().setLevel(logging.ERROR)
import arviz as az
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [6]:
seed: int = sum(map(ord, "stan_hier_beta_binomial"))
rng: np.random.Generator = np.random.default_rng(seed=seed)
sns.set_theme(palette="colorblind")
az.style.use("arviz-darkgrid")
%config InlineBackend.figure_format = "retina"

# Get the home directory
home_directory = os.path.expanduser("~")
# Construct the path to the Quarto project directory 
project_directory = os.path.join(
    home_directory, '_repositories', 'psicometria')

## Introduzione {.unnumbered}

Nel capitolo @sec-change-across-time, utilizzeremo Stan per stimare i parametri di un processo dinamico utilizzando un modello di Markov di primo ordine. In quella discussione, impiegheremo la distribuzione di probabilità categoriale. L'obiettivo di questo capitolo è familiarizzare con questa distribuzione.

La variabile casuale più semplice può assumere solo due stati, comunemente denominati "successo" e "fallimento". Tuttavia, variabili casuali più complesse possono avere più di due stati possibili. In questo capitolo, esploreremo come la teoria della probabilità tratta questi diversi scenari.

## Distribuzioni Bernoulliana e Binomiale

**1. Distribuzione Bernoulliana**:

- La *distribuzione Bernoulliana* (o di Bernoulli) è una distribuzione di probabilità discreta che descrive l'esito di un singolo esperimento dicotomico, cioè un evento che ha due possibili risultati: "successo" (generalmente codificato come 1) o "fallimento" (codificato come 0).
- Un esempio classico è il lancio di una moneta: "testa" (successo) o "croce" (fallimento).
- La probabilità di successo è denotata con $p$ e quella di fallimento con $1 - p$.

**2. Distribuzione Binomiale**:

- La *distribuzione Binomiale* descrive il numero di successi in una sequenza di $N$ esperimenti indipendenti, ognuno dei quali segue una distribuzione di Bernoulli con probabilità di successo $p$.
- Un esempio potrebbe essere il numero di volte in cui si ottiene "testa" quando si lancia una moneta 10 volte.
- La distribuzione binomiale ha due parametri: il numero di prove $N$ e la probabilità di successo $p$.

## Distribuzioni Categoriale e Multinomiale

**1. Distribuzione Categoriale**:

- La *distribuzione Categoriale* è una generalizzazione della distribuzione di Bernoulli per variabili aleatorie con più di due categorie possibili.
- È utilizzata per descrivere l'esito di un singolo esperimento che può avere più di due risultati discreti (categorie), ciascuno con una propria probabilità.
- Un esempio psicologico potrebbe essere una prova in cui un partecipante deve scegliere tra quattro colori diversi (ad esempio, rosso, verde, blu, giallo). Le probabilità associate a ciascun colore rappresentano le probabilità categoriali.
- Se $K$ è il numero di categorie, la distribuzione categoriale è descritta da un vettore di probabilità $\mathbf{p} = (p_1, p_2, \ldots, p_K)$ dove la somma di tutte le probabilità è 1 ($\sum_{i=1}^{K} p_i = 1$).

**2. Distribuzione Multinomiale**:

- La *distribuzione Multinomiale* è una generalizzazione della distribuzione Binomiale per esperimenti con più di due risultati.
- Descrive la distribuzione del numero di successi in ciascuna delle $K$ categorie in una sequenza di $N$ esperimenti indipendenti, dove ogni esperimento segue una distribuzione categoriale con probabilità specifiche per ciascuna categoria.
- Un esempio psicologico potrebbe essere una serie di 20 prove in cui un partecipante sceglie uno dei quattro colori in ogni prova. La distribuzione multinomiale descrive il numero di volte in cui ciascun colore è scelto.

## Relazioni tra Distribuzioni

In sintesi, possiamo descrivere le relazioni tra le distribuzioni nel modo seguente.

- *Bernoulliana* e *Binomiale*: sono appropriate per eventi dicotomici (due possibili risultati). Una singola prova è modellata da una distribuzione Bernoulliana; una serie di prove da una distribuzione Binomiale.
- *Categoriale* e *Multinomiale*: sono appropriate per eventi con più di due risultati. Una singola prova è modellata da una distribuzione categoriale; una serie di prove da una distribuzione Multinomiale.

## Esempio con la Distribuzione Binomiale

Un risultato consolidato nella letteratura psicologica è che le persone con disturbi emotivi, come il disturbo d'ansia sociale (SAD), il disturbo d'ansia generalizzata (GAD) e la depressione, mostrano una tendenza costante, o bias, a generare interpretazioni negative di materiali ambigui. Questo è diverso rispetto alle persone senza disturbi emotivi, che tendono, in generale, a fornire interpretazioni positive agli stimoli ambigui (@hirsch2016resolving).

Immaginiamo un compito psicologico in cui a ciascun partecipante di un campione di $N$ individui depressi viene chiesto di completare una singola prova. Viene presentata un'immagine ambigua e il partecipante deve scegliere tra due emozioni: felice o triste. L'uso di una singola prova per ciascun soggetto può essere giustificato per evitare di allertare il soggetto rispetto alle caratteristiche richieste dal compito, presentando questa prova cruciale all'interno di una serie di altre prove diverse che fungono da "filler". 

Negli esperimenti effettivi in quest'area di ricerca si utilizzano stimoli come omofoni (ciascuno con un significato negativo e uno non negativo, ad esempio "die/dye") oppure compiti di comprensione del testo in cui ogni set di frasi include una frase ambigua che ha un significato negativo e uno non correlato alla depressione (ad esempio, @mogg2006interpretation).

Nell'esempio presente, immaginiamo che i dati raccolti siano costituiti da $N$ osservazioni, dove 1 indica un'interpretazione negativa e 0 indica un'interpretazione non negativa. La variabile casuale che rappresenta l'esito dell'esperimento è Bernoulliana, poiché ogni prova può avere solo due risultati possibili (negativo o non negativo). Lo scopo dell'inferenza è stimare il parametro $\theta$, che rappresenta la probabilità di un'interpretazione negativa.

Ecco un esempio di codice Stan che modella questo scenario:

```stan
data {
  int<lower=1> N; // Numero di partecipanti
  int<lower=0, upper=1> y[N]; // Risultati delle prove (valori 0 e 1)
}
parameters {
  real<lower=0, upper=1> theta; // Probabilità di interpretazione negativa
}
model {
  y ~ bernoulli(theta); // Likelihood: distribuzione Bernoulliana per ciascuna prova
}
generated quantities {
  int y_pred; // Predizione per una nuova prova
  y_pred = bernoulli_rng(theta);
}
```

Spiegazione del Codice Stan:

1. **Dati**:

   - `N`: il numero totale di partecipanti.
   - `y`: un array di lunghezza $N$ che contiene i risultati delle prove per ciascun partecipante, con valori 0 (interpretazione non negativa) o 1 (interpretazione negativa).

2. **Parametri**:

   - `theta`: un parametro che rappresenta la probabilità di un'interpretazione negativa, vincolato tra 0 e 1.

3. **Modello**:

   - `y ~ bernoulli(theta)`: specifica che ogni osservazione $y[i]$ segue una distribuzione Bernoulliana con parametro $\theta$. Questa è la *likelihood* del modello, che indica come i dati osservati sono generati dato il parametro $\theta$.

4. **Quantità Generate**:

   - `y_pred`: una variabile che rappresenta la predizione di un nuovo esito sulla base delle probabilità posteriori stimate. Utilizza la funzione `bernoulli_rng(theta)` per generare un'osservazione simulata secondo la distribuzione Bernoulliana con parametro $\theta$.

Questo modello Stan permette di stimare la probabilità $\theta$ che un individuo con depressione interpreti un'ambiguità in modo negativo, sulla base dei dati osservati.

## Esempio con la Distribuzione Categoriale

Per illustrare il modello categoriale, possiamo partire dallo scenario descritto in precedenza, con una piccola modifica. Immaginiamo che venga presentata un'immagine ambigua al partecipante, ma questa volta il partecipante deve scegliere tra quattro emozioni possibili: felice, triste, arrabbiato o neutrale. Ogni scelta rappresenta un risultato categoriale, poiché ci sono più di due possibili risposte.

In questo caso, la variabile casuale $X$ che rappresenta la risposta del soggetto può assumere quattro valori distinti, corrispondenti alle quattro emozioni. La funzione di massa di probabilità (PMF) di $X$ è descritta da un **simplex** a 4 dimensioni, il che significa che ogni probabilità è non negativa e la somma delle probabilità di tutte le categorie è pari a uno. Formalmente, se $\theta = (\theta_1, \theta_2, \theta_3, \theta_4)$ rappresenta il vettore delle probabilità per ciascuna emozione, allora per $x \in \{1, 2, 3, 4\}$, la probabilità categoriale è data da:

$$
P(X = x) = \theta_x,
$$

dove $\theta_x$ rappresenta la probabilità di ciascun risultato $x$.

Ad esempio, supponiamo che le probabilità per le quattro emozioni siano:

- $P(X = 1) = 0.2$ (felice),
- $P(X = 2) = 0.4$ (triste),
- $P(X = 3) = 0.1$ (arrabbiato),
- $P(X = 4) = 0.3$ (neutrale).

In questo caso, il vettore delle probabilità è $\theta = (0.2, 0.4, 0.1, 0.3)$, che rappresenta una distribuzione categoriale in cui le probabilità di ogni emozione sommano a 1, come richiesto da un simplex.

Ecco come modellare questo scenario utilizzando Stan:

```stan
data {
  int<lower=1> N; // Numero di partecipanti
  array[N] int<lower=1, upper=4> y;  // Risultati delle prove (valori da 1 a 4)
}
parameters {
  simplex[4] theta; // Vettore delle probabilità categoriali per le quattro emozioni
}
model {
  y ~ categorical(theta); // Likelihood: distribuzione categoriale per ciascuna prova
}
generated quantities {
  int y_pred; // Predizione per una nuova prova
  y_pred = categorical_rng(theta);
}
```

Nel codice precedente 

1. **Dati**:

   - `N`: il numero totale di partecipanti.
   - `y`: un array di lunghezza $N$ che contiene i risultati delle prove per ciascun partecipante, dove ogni valore può essere 1 (felice), 2 (triste), 3 (arrabbiato), o 4 (neutrale).

2. **Parametri**:

   - `theta`: un vettore di probabilità di tipo `simplex[4]`, che rappresenta la distribuzione delle probabilità per le quattro emozioni. Il tipo `simplex` garantisce che tutte le probabilità siano non negative e che la loro somma sia pari a 1.

  Quando $\theta$ è dichiarato come `simplex[4]` senza una specifica dichiarazione di prior nel blocco `model`, Stan assume automaticamente una distribuzione a priori uniforme sulla superficie del simplex. In altre parole, ogni possibile configurazione di $\theta$ che soddisfa le condizioni di un `simplex` (ovvero, tutte le componenti sono non negative e sommano a 1) ha la stessa probabilità a priori. Questo significa che, prima di vedere i dati, tutte le combinazioni di valori di $\theta$ che sommano a 1 sono considerate ugualmente probabili. Non c'è alcun bias a priori che favorisca una particolare configurazione di probabilità tra le quattro emozioni (felice, triste, arrabbiato, neutrale). Quindi, nel modello Stan, la distribuzione a priori su $\theta$ è effettivamente uniforme sul simplex, riflettendo una posizione a priori non informativa.

3. **Modello**:

   - `y ~ categorical(theta)`: specifica che ogni osservazione $y[i]$ segue una distribuzione categoriale con parametro $\theta$. Questa è la *likelihood* del modello, che descrive come i dati osservati sono generati dato il vettore di probabilità $\theta$.

4. **Quantità Generate**:

   - `y_pred`: una variabile che rappresenta la predizione di un nuovo esito sulla base delle probabilità posteriori stimate. Utilizza la funzione `categorical_rng(theta)` per generare un'osservazione simulata secondo la distribuzione categoriale con parametro $\theta$.

Questo modello Stan consente di stimare le probabilità associate a ciascuna emozione scelta dai partecipanti in risposta a un'immagine ambigua.

## Simulazione

Vediamo ora come implementare il modello categoriale in pratica. Iniziamo simulando dei dati dalla distribuzione categoriale.

In [8]:
# Definire i valori possibili e le probabilità associate
valori = [1, 2, 3, 4]
probabilita = [0.2, 0.4, 0.1, 0.3]

# Generare 50 valori casuali dalla distribuzione categoriale
y = np.random.choice(valori, size=50, p=probabilita)
print(y)

[3 2 2 2 1 1 2 2 4 2 4 1 3 2 1 4 2 2 2 2 4 3 3 1 3 4 1 4 1 4 2 4 4 2 1 2 4
 3 2 2 2 1 1 4 2 4 2 4 2 4]


Inseriamo i dati in un dizionario come richiesto da Stan.

In [9]:
stan_data = {
    "N": len(y),
    "y": y.tolist()
}

Importiamo il codice Stan che abbiamo discusso in precedenza.

In [13]:
stan_file = os.path.join(project_directory, "stan", "categorical_model.stan")

Compiliamo il modello.

In [14]:
model = CmdStanModel(stan_file=stan_file)

Eseguiamo il campionamento MCMC utilizzando i dati simulati.

In [15]:
fit = model.sample(
    data=stan_data,
    iter_warmup=2000, 
    iter_sampling=2000,
    seed=42, 
    chains=4,
    show_progress=False, 
    show_console=False
)

Esaminiamo le stime a posteriori dei parametri.

In [18]:
az.summary(fit, var_names="theta", hdi_prob=0.95).round(2)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
theta[0],0.20,0.05,0.10,0.31,0.0,0.0,15575.0,10516.0,1.0
theta[1],0.39,0.06,0.27,0.52,0.0,0.0,15457.0,12342.0,1.0
theta[2],0.13,0.04,0.05,0.22,0.0,0.0,14350.0,10458.0,1.0
theta[3],0.28,0.06,0.16,0.40,0.0,0.0,15702.0,11989.0,1.0


## Interpretazione dei Risultati

Nel contesto di questo esempio, immaginiamo un gruppo di pazienti depressi che giudicano delle immagini ambigue, assegnando loro un'emozione tra "felice", "triste", "arrabbiato" o "neutrale". I parametri $\theta[0]$, $\theta[1]$, $\theta[2]$ e $\theta[3]$ rappresentano le stime a posteriori delle probabilità che un partecipante scelga rispettivamente ciascuna di queste emozioni.

### Interpretazione Specifica dei Parametri

- **$\theta[0]$: Emozione "felice"**  
  La stima a posteriori della probabilità che un partecipante scelga l'emozione "felice" ($P(X = 1)$) è 0.20, con un intervallo di credibilità al 95% (HDI) che va da 0.10 a 0.31. Questo suggerisce che, per questo gruppo di pazienti depressi, c'è una probabilità relativamente bassa di interpretare un'immagine ambigua come "felice".

- **$\theta[1]$: Emozione "triste"**  
  La stima a posteriori della probabilità che un partecipante scelga l'emozione "triste" ($P(X = 2)$) è 0.39, con un HDI al 95% tra 0.27 e 0.52. Questo valore più alto rispetto alle altre emozioni indica che i pazienti depressi nel campione hanno una tendenza maggiore a interpretare immagini ambigue come "tristi". Questo risultato è coerente con le aspettative teoriche secondo cui le persone con depressione hanno una propensione a percepire situazioni ambigue in modo più negativo.

- **$\theta[2]$: Emozione "arrabbiato"**  
  La stima a posteriori della probabilità che un partecipante scelga l'emozione "arrabbiato" ($P(X = 3)$) è 0.13, con un HDI al 95% che va da 0.05 a 0.22. Questa probabilità relativamente bassa suggerisce che interpretare un'immagine come "arrabbiato" non è un'interpretazione comune tra i pazienti depressi nel campione.

- **$\theta[3]$: Emozione "neutrale"**  
  La stima a posteriori della probabilità che un partecipante scelga l'emozione "neutrale" ($P(X = 4)$) è 0.28, con un HDI al 95% tra 0.16 e 0.40. Una probabilità del 28% indica che un'interpretazione "neutrale" delle immagini ambigue è relativamente comune, ma non dominante, tra i pazienti depressi.

In sintesi, queste stime posteriori forniscono una chiara indicazione delle tendenze emotive dei partecipanti depressi nel contesto di interpretazioni di stimoli ambigui. L'alta probabilità di scegliere "triste" e le basse probabilità associate alle altre emozioni sono coerenti con la letteratura esistente che indica un bias negativo nelle interpretazioni tra persone con depressione. 

In questo esempio simulato, le stime posteriori di $\theta$ riflettono accuratamente i valori delle probabilità utilizzati per generare i dati, dimostrando che anche con un numero limitato di dati, il modello bayesiano è capace di recuperare le distribuzioni delle probabilità a posteriori in modo efficace. 

## Considerazioni Conclusive

L'obiettivo di questo breve capitolo era fornire un esempio della distribuzione categoriale, che rappresenta un'estensione della distribuzione di Bernoulli per il caso in cui vi siano più di due categorie possibili. Abbiamo mostrato come utilizzare Stan per eseguire inferenze su questa distribuzione, permettendo di stimare le probabilità associate a ciascuna categoria basandosi sui dati osservati. Questo approccio è utile in vari contesti psicologici in cui le risposte dei partecipanti non si limitano a scelte dicotomiche, ma comprendono più opzioni.

## Informazioni sull'Ambiente di Sviluppo {.unnumbered}

In [36]:
%load_ext watermark
%watermark -n -u -v -iv -w -m -p cmdstanpy 

Last updated: Sat Jul 27 2024

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

cmdstanpy: 1.2.4

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

arviz     : 0.18.0
pandas    : 2.2.2
matplotlib: 3.9.1
numpy     : 1.26.4
requests  : 2.32.3
cmdstanpy : 1.2.4
logging   : 0.5.1.2

Watermark: 2.4.3

